 Copyright © Sorbonne University.

 This source code is licensed under the MIT license found in the
 LICENSE file in the root directory of this source tree.

# Outlook

In this notebook, you will code a naive actor-critic algorithm in the tabular case. Then you will tune it using grid search and Bayesian optimization, potentially using the [optuna](https://optuna.readthedocs.io/en/stable/) library.
Finally, you will get the best hyper-parameters obtained with both methods and perform a statistical test to see if there is a statistically significant difference between these methods and with respect to naive hyper-parameter values.

## Install libraries

In [1]:
# Installs the necessary Python and system libraries
try:
    from easypip import easyimport, easyinstall, is_notebook
except ModuleNotFoundError as e:
    get_ipython().run_line_magic("pip", "install 'easypip>=1.2.0'")
    from easypip import easyimport, easyinstall, is_notebook

easyinstall("swig")
easyinstall("bbrl>=0.2.2")
easyinstall("bbrl_gymnasium>=0.2.0")
easyinstall("tensorboard")
easyinstall("moviepy")
easyinstall("box2d-kengz")
easyinstall("optuna")
easyinstall("gymnasium")
easyinstall("mazemdp")

import numpy as np
import os
from typing import List

import hydra
import optuna
import yaml
from omegaconf import OmegaConf, DictConfig

# For visualization
os.environ["VIDEO_FPS"] = "5"
if not os.path.isdir("./videos"):
    os.mkdir("./videos")

from IPython.display import Video

In [2]:
import torch
import torch.nn as nn

In [3]:
import gymnasium as gym

from bbrl.utils.chrono import Chrono

import matplotlib
import matplotlib.pyplot as plt

from mazemdp.toolbox import sample_categorical
from mazemdp.mdp import Mdp
from bbrl_gymnasium.envs.maze_mdp import MazeMDPEnv
from gymnasium.wrappers.monitoring.video_recorder import VideoRecorder
from functools import partial

matplotlib.use("TkAgg")

Matplotlib backend: module://matplotlib_inline.backend_inline


# Step 1: Coding the naive Actor-critic algorithm

We consider the naive actor-critic algorithm with a categorical policy.
The algorithm learns a critic with the standard temporal difference mechanism
using a learning rate $\alpha_{critic}$.

We consider a value-based critic $V(s)$. The extension to an action value function $Q(s,a)$ is straightforward.

To update the critic, the algorithm computes the temporal difference error:

$$\delta_t = r(s_t, a_t) + \gamma V^{(n)}(s_{t+1})-V^{(n)}(s_t).$$

Then it applies it to the critic:

$$V^{(n+1)}(s_t) = V^{(n)}(s_t) + \alpha_{critic} \delta_t.$$

To update the actor, the general idea is the same, using the temporal difference error with another learning rate $\alpha_{actor}$.

However, naively applying the same learning rule would not ensure that the probabilities of all actions in a state sum to 1.
Besides, when the temporal difference error $\delta_t$ is negative, it may happen that the probability of an action gets negative or null, which raises an issue when applying renormalization.

So, instead of applying the naive rule, we apply the following one:
$$ 
\pi_{temp}(a_t|s_t) =  \begin{cases}
\pi^{(i)}(a_t|s_t) + \alpha_{actor} \delta_t & \mathrm{if } \pi^{(i)}(a_t|s_t) + \alpha_{actor} \delta_t > 10^{-8}\\
10^{-8} & \mathrm{otherwise.} \\
\end{cases}
$$

Then we can apply renormalization so that the probabilities of actions still sum to 1, with
$$
\forall a, \pi^{(i+1)}(a|s_t) = \frac{\pi_{temp}^{(i+1)}(a|s_t)} {\sum_{a'} \pi_{temp}^{(i+1)}(a'|s_t)}
$$ with
$$ 
\pi_{temp}^{(i+1)}(a|s_t) =  \begin{cases}
\pi_{temp}(a|s_t) & \mathrm{if } a = a_t\\
\pi^{(i)}(a|s_t) & \mathrm{otherwise.} \\
\end{cases}
$$

## Exercise 1

### 1. Code the naive actor-critic algorithm as specified above.

Some hints:

- a good idea to build this code it to take inspiration from the code of Q-learning, to add an actor (a categorical policy), both learning rates,
and to take care about the renormalization function.

- for the next steps of this lab, having a function to repeatedly call your actor-critic algorithm and save the learning trajectories and
norms of the value function is a good idea.

In [4]:
import gymnasium as gym
import numpy as np 

# Environment with 20% of walls and no negative reward when hitting a wall
env = gym.make(
    "MazeMDP-v0",
    kwargs={"width": 4, "height": 3, "ratio": 0.2, "hit": 0.0},
    render_mode="human",
)
env.reset()
env.unwrapped.init_draw("The maze")

       [-1,  2,  5,  8],
       [ 0,  3,  6,  9]]), t…

In [17]:
def get_policy_from_actor(actor: np.ndarray) -> np.ndarray:
    return np.argmax(actor,axis=1)

def actor_critic(
    mdp: MazeMDPEnv,
    nb_episodes: int = 20,
    timeout: int = 50,
    alpha_critic : float = 0.5,
    alpha_actor : float=0.5,
    render: bool = True,
) -> tuple[np.ndarray, List[float]]:
    # Initialize the state-action value function
    # alpha is the learning rate
    v = np.zeros(mdp.unwrapped.nb_states)  # initial state value v
    a_probs = np.ones((mdp.unwrapped.nb_states, mdp.action_space.n))/mdp.action_space.n 
    v_min = np.zeros(mdp.unwrapped.nb_states)
    v_list = []
    time_list = []
    time_list_terminated = []

    # Run learning cycle
    mdp.timeout = timeout  # episode length

    if render:
        mdp.init_draw("Actor Critic")

    for _ in range(nb_episodes):
        # Draw the first state of episode i using a uniform distribution over all the states
        x, _ = mdp.reset(uniform=True)
        cpt = 0

        terminated = False
        truncated = False

        while not (terminated or truncated):
            if render:
                mdp.draw_v_pi(v, get_policy_from_actor(a_probs))

            # Draw an action using a soft-max policy
            u = sample_categorical(a_probs[x])
            y, r, terminated, truncated, _ = mdp.step(u)
            # Update the state-action value function with q-Learning
            delta = r + mdp.gamma*v[y]*(1-terminated) - v[x]
            v[x] = v[x] + alpha_critic*delta

            temp = a_probs[x,u] + alpha_actor*delta
            a_probs[x, u] = temp if temp > 1e-8 else 1e-8
            a_probs[x] = a_probs[x]/a_probs[x].sum()

            x = y
            cpt = cpt + 1

        v_list.append(np.linalg.norm(np.maximum(v, v_min)))
        if terminated:
            time_list.append(cpt)

    if render:
        mdp.current_state = 0
        mdp.draw_v_pi(v, get_policy_from_actor(a_probs))

    return a_probs, v, v_list, time_list

In [34]:
NB_EPISODES = 100
TIMEOUT = 50
ALPHA_CRITIC = 0.5
ALPHA_ACTOR = 0.5

a, v, v_list, time_list = actor_critic(
    env, alpha_critic=ALPHA_CRITIC,alpha_actor=ALPHA_ACTOR, nb_episodes=NB_EPISODES, timeout=TIMEOUT
)

/mnt/c/Cours-Sorbonne/M2/UE_DEEP/.venv/lib/python3.10/site-packages/gymnasium/core.py:311: UserWarning: WARN: env.init_draw to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.init_draw` for environment variables or `env.get_wrapper_attr('init_draw')` that will search the reminding wrappers.
  logger.warn(


       [-1,  2,  5,  8],
       [ 0,  3,  6,  9]]), t…

/mnt/c/Cours-Sorbonne/M2/UE_DEEP/.venv/lib/python3.10/site-packages/gymnasium/core.py:311: UserWarning: WARN: env.draw_v_pi to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.draw_v_pi` for environment variables or `env.get_wrapper_attr('draw_v_pi')` that will search the reminding wrappers.
  logger.warn(
/mnt/c/Cours-Sorbonne/M2/UE_DEEP/.venv/lib/python3.10/site-packages/gymnasium/core.py:311: UserWarning: WARN: env.gamma to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.gamma` for environment variables or `env.get_wrapper_attr('gamma')` that will search the reminding wrappers.
  logger.warn(


In [35]:
a

array([[1.27466826e-01, 1.13146645e-01, 6.44820172e-01, 1.14566357e-01],
       [1.21059986e-01, 5.59948268e-01, 1.97920524e-01, 1.21071222e-01],
       [8.11309572e-02, 2.89931614e-01, 6.28937418e-01, 1.00907046e-08],
       [9.42029902e-09, 1.21736141e-01, 8.78263841e-01, 8.78808368e-09],
       [1.09540099e-01, 6.53464805e-01, 1.09815316e-01, 1.27179780e-01],
       [9.46836300e-09, 9.83306445e-01, 1.66935353e-02, 1.00891060e-08],
       [7.06451818e-09, 9.82993958e-09, 9.99999978e-01, 5.32859662e-09],
       [7.38572615e-02, 8.83669139e-01, 2.46467713e-02, 1.78268286e-02],
       [9.34071830e-09, 9.99999973e-01, 9.18322697e-09, 8.35304460e-09],
       [1.04856564e-01, 4.16384724e-01, 1.34246550e-01, 3.44512162e-01]])

### 2. Provide a plot function

Your plot function should show the evolution through time of number of steps the agent takes to find the reward in the maze.
If your algorithm works, this number of steps should decrease through time.

Your plot function should also show a mean and a standard deviation (or some more advanced statistics) over a collection of learning runs.

In [68]:
import numpy as np 
import seaborn as sns 


def moving_average(data, window_size):
    return np.convolve(data, np.ones(window_size)/window_size, mode='valid')


def plot_evolution(time_list):
    mean = np.round(np.mean(time_list), 2)
    std = np.round(np.std(time_list), 2)
    q1 = np.percentile(time_list, 25)
    q3 = np.percentile(time_list, 75)
    median = np.median(time_list)
    max = np.max(time_list)
    min = np.min(time_list)

    time_list_smoothed = moving_average(time_list, 10)

    plt.style.use("ggplot")
    plt.figure(figsize=(14, 7))

    # Plot des étapes par épisode
    plt.subplot(121)
    plt.plot(time_list, label="Steps per episode", color='blue', alpha=0.6)
    plt.plot(time_list_smoothed, label="Moving average of steps per episode", color='red', linewidth=2)
    plt.legend()
    plt.title("Number of Steps per Episode")
    plt.xlabel("Episode")
    plt.ylabel("Steps")
    plt.grid(True)

    # Boxplot des étapes par épisode
    plt.subplot(122)
    plt.boxplot(time_list, vert=False)
    plt.title(f"Box plot of steps per episode")
    plt.xlabel("Steps")
    plt.grid(True)

    handles, labels = plt.gca().get_legend_handles_labels()
    handles.extend([
        plt.Line2D([0], [0], color='yellow', lw=2),
        plt.Line2D([0], [0], color='yellow', lw=2),
        plt.Line2D([0], [0], color='green', lw=2),
        plt.Line2D([0], [0], color='purple', lw=2),
        plt.Line2D([0], [0], color='red', lw=2),
        plt.Line2D([0], [0], color='blue', lw=2),
        plt.Line2D([0], [0], color='blue', lw=2)
    ])
    labels.extend([
        f'Max : {max} steps', f'Min : {min} steps', 
        f'Mean : {mean} steps', f'Std : {std} steps', 
        f'Q1 : {q1} steps', f'Q3 : {q3} steps', 
        f'Median : {median} steps'
    ])
    
    plt.legend(handles, labels, loc='upper right')

    plt.tight_layout()
    plt.show()


plot_evolution(time_list)

## Actor-critic hyper-parameters

To represent the hyper-parameters of the experiments performed in this notebook, we suggest using the dictionary below.
This dictionary can be read using omegaconf.
Using it is not mandatory.
You can also change the value of hyper-parameters or environment parameters at will.

In [9]:
ac_params = {
    "save_curves": False,
    "save_heatmap": True,
    "mdp": {
        "name": "MazeMDP-v0",
        "width": 5,
        "height": 5,
        "ratio": 0.2,
        "render_mode": "rgb_array",
        },
        
    "log_dir": "./tmp",
    "video_dir": "./tmp/videos",

    "nb_episodes": 100,
    "timeout": 200,
    "render": False, # True, # 
    "nb_repeats": 5,

    "alpha_critic": 0.5,
    "alpha_actor": 0.5,
}

### 3. Test your code

Once everything looks OK, save the obtained plot for your lab report

In [10]:
# To be completed...

assert False, 'Not implemented yet'


AssertionError: Not implemented yet

# Step 2: Tuning hyper-parameters

In this part, you have to optimize two hyper-parameters of the actor-critic algorithm, namely the actor and critic learning rates.
You have to do so using a simple grid search method and some Bayesian optimization method.
For the latter, we suggest using the default sampler from [optuna](https://optuna.readthedocs.io/en/stable/).
Follow the above link to understand how optuna works.
Note that it also supports grid search and many other hyper-parameters tuning algorithms.

You should make sure that the hyper-parameters tuning algorithms that you compare benefit from the same training budget
We suggest 400 training runs overall for each method,
which means 20 values each for the actor and the critic learning rates in the case of grid search.

## Exercise 2

### 1. Perform hyper-parameters tuning with two algorithms as suggested above.

### 2. Provide a "heatmap" of the norm of the value function given the hyper-parameters, after training for each pair of hyper-parameters.

### 3. Collect the value of the best hyper-parameters found with each algorithm. You will need them for Step 3.

### 4. Include in your report the heatmaps and the best hyper-parameters found for each method.

In [ ]:
# To be completed...

assert False, 'Not implemented yet'


# Step 3: Statistical tests

Now you have to compare the performance of the actor-critic algorithm tuned
with all the best hyper-parameters you found before, using statistical tests.

The functions below are provided to run Welch's T-test over learning curves.
They have been adapted from a github repository: https://github.com/flowersteam/rl_stats
You don't need to understand them in detail (though it is always a good idea to try to understand more code).

In [ ]:
from scipy.stats import ttest_ind
import bootstrapped.bootstrap as bs
import bootstrapped.compare_functions as bs_compare
import bootstrapped.stats_functions as bs_stats

In [ ]:
def compute_central_tendency_and_error(id_central, id_error, sample):

    try:
        id_error = int(id_error)
    except:
        pass

    if id_central == "mean":
        central = np.nanmean(sample, axis=1)
    elif id_central == "median":
        central = np.nanmedian(sample, axis=1)
    else:
        raise NotImplementedError

    if isinstance(id_error, int):
        low = np.nanpercentile(sample, q=int((100 - id_error) / 2), axis=1)
        high = np.nanpercentile(sample, q=int(100 - (100 - id_error) / 2), axis=1)
    elif id_error == "std":
        low = central - np.nanstd(sample, axis=1)
        high = central + np.nanstd(sample, axis=1)
    elif id_error == "sem":
        low = central - np.nanstd(sample, axis=1) / np.sqrt(sample.shape[0])
        high = central + np.nanstd(sample, axis=1) / np.sqrt(sample.shape[0])
    else:
        raise NotImplementedError

    return central, low, high

In [ ]:
def run_test(test_id, data1, data2, alpha=0.05):
    """
    Compute tests comparing data1 and data2 with confidence level alpha
    :param test_id: (str) refers to what test should be used
    :param data1: (np.ndarray) sample 1
    :param data2: (np.ndarray) sample 2
    :param alpha: (float) confidence level of the test
    :return: (bool) if True, the null hypothesis is rejected
    """
    data1 = data1.squeeze()
    data2 = data2.squeeze()
    n1 = data1.size
    n2 = data2.size

    # perform Welch t-test":
    _, p = ttest_ind(data1, data2, equal_var=False)
    return p < alpha

This last function was adapted for the lab.

In [ ]:
def perform_test(perf1, perf2, name1, name2, sample_size=20, downsampling_fact=5, confidence_level=0.01):

    perf1 = perf1.transpose()
    perf2 = perf2.transpose()
    nb_datapoints = perf1.shape[1]
    nb_steps = perf1.shape[0]

    legend = [name1, name2]

    # what do you want to plot ?
    id_central = 'mean' # "median"  # 
    id_error = 80  # (percentiles), also: 'std', 'sem'

    test_id = "Welch t-test"  # recommended
    
    sample1 = perf1[:, np.random.randint(0, nb_datapoints, sample_size)]
    sample2 = perf2[:, np.random.randint(0, nb_datapoints, sample_size)]

    steps = np.arange(0, nb_steps, downsampling_fact)
    sample1 = sample1[steps, :]
    sample2 = sample2[steps, :]

    # test
    sign_diff = np.zeros([len(steps)])
    for i in range(len(steps)):
        sign_diff[i] = run_test(
            test_id, sample1[i, :], sample2[i, :], alpha=confidence_level
        )

    central1, low1, high1 = compute_central_tendency_and_error(
        id_central, id_error, sample1
    )
    central2, low2, high2 = compute_central_tendency_and_error(
        id_central, id_error, sample2
    )

    # plot
    _, ax = plt.subplots(1, 1, figsize=(20, 10))
    lab1 = plt.xlabel("training steps")
    lab2 = plt.ylabel("performance")

    plt.plot(steps, central1, linewidth=10)
    plt.plot(steps, central2, linewidth=10)
    plt.fill_between(steps, low1, high1, alpha=0.3)
    plt.fill_between(steps, low2, high2, alpha=0.3)
    leg = ax.legend(legend, frameon=False)

    # plot significative difference as dots
    idx = np.argwhere(sign_diff == 1)
    y = max(np.nanmax(high1), np.nanmax(high2))
    plt.scatter(steps[idx], y * 1.05 * np.ones([idx.size]), s=100, c="k", marker="o")

    # style
    for line in leg.get_lines():
        line.set_linewidth(10.0)
    ax.spines["top"].set_linewidth(5)
    ax.spines["right"].set_linewidth(5)
    ax.spines["bottom"].set_linewidth(5)
    ax.spines["left"].set_linewidth(5)

    plt.savefig(
        f"./{name1}_{name2}.png", bbox_extra_artists=(leg, lab1, lab2), bbox_inches="tight", dpi=100
    )
    # plt.show()

## Exercise 3

As hyper-parameters, you will use:

- naive tuning, that is a pair (0.5, 0.5) for the actor and critic learning rates,
- the best hyper-parameters you found with the different tuning algorithms you used before.

### 1. For each set of hyper-parameters, collect a large dataset of learning curves.

We suggest using 150 training episodes.

### 2. Perform statistical comparisons

- Take two datasets of learning curves obtained with the hyper-parameters sets that you found with different tuning algorithms.
- Use the ``` perform_test(...)``` function to compare each possible pair of sets.

You should obtain an image for each pair you have tried.
In this image, black dots signal the time step where there is a statistically significant difference between two learning curves.

 ### 3. Conclude.

In [ ]:
# To be completed...

assert False, 'Not implemented yet'


# Lab report

Your report should contain:
- your source code (probably this notebook), do not forget to put your names on top of the notebook,
- in a separate pdf file with your names in the name of the file:
    + a detailed enough description of the choices you have made: the parameters you have set, the libraries you have used, etc.,
    + the heatmaps obtained using the hyper-parameters tuning algorithms that you have used,
    + the figures resulting from performing Welch's T-test using the best hyper-parameters from the above approaches,
    + your conclusion from these experiments.

Beyond the elements required in this report, any additional studies will be rewarded.
For instance, you can try using a Q-function as critic, using random search as hyper-parameters tuning algorithm,
using more challenging environments, etc.